# Chapter 3 에서 배울 내용

1. **데이터 로더 제작**  
   - 모델 학습을 위한 데이터 로더 설계 및 구현  
   - 분자 데이터 분류를 위한 사전 지식 이해

---
2. **모델 제작**  
    - 분자 특성 분류/회귀 예측을 위한 모델 아키텍처 설계 
    - MLP, CNN, RNN, GNN 등 다양한 모델 활용 방법  
    - 입력 데이터 처리 및 모델에 맞는 출력 형식 설정

---
3. **학습 및 평가**
   - 모델 학습을 위한 손실 함수와 최적화 알고리즘 설정  
   - 교차 검증, 정확도, 정밀도, 재현율, F1 스코어 등 다양한 평가 지표 활용  
   - 과적합 방지를 위한 기법 (예: Dropout, 정규화, 데이터 증강 등)
   - 모델 성능 평가 
    
---
4. **결과 저장**
    - 학습 로그 및 평가 결과 기록  
    - 결과 시각화 및 분석 도구 사용 (예: Loss/Accuracy 그래프, confusion matrix 등)
    - 모델을 실험적 환경에서 재사용할 수 있도록 파일 포맷으로 저장 (예: `.pt`, `.h5` 등) 
    
---

In [29]:
import os
import numpy as np
import pandas as pd

from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, rdDepictor, rdDistGeom, MACCSkeys, rdMolDescriptors

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU, Conv1d
from torch.utils.data import Dataset
from torch_geometric import utils as pyg_utils
from torch_geometric.data import InMemoryDataset, download_url, extract_gz, Data, Batch
from torch_geometric.loader import DataLoader
import torch_geometric.nn as gnn
from torch_geometric.nn import GCNConv, GINConv, GATConv, global_max_pool as gmp
from torch_geometric.nn import global_add_pool

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score
from tabulate import tabulate

## 1) 데이터 로더 제작

데이터 로더(Data Loader)는 머신러닝/딥러닝 모델 학습에서 데이터를 효율적으로 불러오고 처리하기 위한 도구다. 

일반적으로 `PyTorch`와 같은 프레임워크에서 제공되며, 대규모 데이터셋을 다룰 때 데이터를 배치 단위로 나누거나 전처리 작업을 자동화하는 데 사용된다.

데이터 로더를 사용하는 이유는 크게 다음과 같다.

---
1. **미니배치 생성**  
   - 데이터를 미니배치 단위로 나누어 처리하여 학습 속도와 메모리 효율성을 높임
   - 배치 크기가 작으면 메모리 사용량이 적으나, 학습 시간이 길어질 수 있음
   - 배치 크기가 크면 학습 속도가 빨라질 수 있으나, 메모리 사용량이 증가함

2. **데이터 셔플링**  
   - 학습 데이터 순서를 매 에포크마다 변경해 모델의 일반화 성능을 향상
   - 학습 시 특정 순서나 패턴으로 인해 모델이 편향되거나 과적합되지 않도록 하기 위해 사용됨.
   - 검증 및 테스트 데이터에서는 **shuffle을 사용하지 않음** (결과 재현성을 위해)

In [2]:
def loader_dataset(data_list, batch_size, shuffle=False):
    """
    DataLoader로 변경
    """
    collate = Batch.from_data_list(data_list)
    loader = DataLoader(data_list, batch_size=batch_size, collate_fn=collate, shuffle=shuffle)

    return loader

본 튜토리얼에서는 ESOL 데이터셋을 예시로 사용한다.

먼저, Chapter2에서 저장한 데이터들을 불러와 data loader를 생성한다.

In [4]:
def get_dataloader(dataset_name, data_type='Graph', batch_size=256):
    data_path = f'dataset/{dataset_name}/processed/'
    train_dataset = torch.load(os.path.join(data_path, f'{dataset_name}_{data_type}_train.pt'))
    valid_dataset = torch.load(os.path.join(data_path, f'{dataset_name}_{data_type}_valid.pt'))
    test_dataset = torch.load(os.path.join(data_path, f'{dataset_name}_{data_type}_test.pt'))    

    if data_type == 'Descriptors':  # Descriptors 데이터를 처리할 경우 scaling 필요
        # Step 1: Descriptors 데이터에서 x 값들만 추출
        raw_features_train = [data.x.view(-1).numpy() for data in train_dataset]
        raw_features_valid = [data.x.view(-1).numpy() for data in valid_dataset]
        raw_features_test = [data.x.view(-1).numpy() for data in test_dataset]
        # print(raw_features_train.shape)

        # Step 2: 스케일링을 위한 StandardScaler 적용
        scaler = StandardScaler()
        scaled_features_train = scaler.fit_transform(raw_features_train)  # 학습 데이터에 맞춰 스케일링
        scaled_features_valid = scaler.transform(raw_features_valid)  # 검증 데이터는 fit하지 않고 transform만 적용
        scaled_features_test = scaler.transform(raw_features_test)  # 테스트 데이터도 동일

        # Step 3: 스케일링된 데이터를 원본 데이터셋에 반영
        for i, data in enumerate(train_dataset):
            data.x = torch.tensor(scaled_features_train[i], dtype=torch.float).view(1, -1)
        for i, data in enumerate(valid_dataset):
            data.x = torch.tensor(scaled_features_valid[i], dtype=torch.float).view(1, -1)
        for i, data in enumerate(test_dataset):
            data.x = torch.tensor(scaled_features_test[i], dtype=torch.float).view(1, -1)       

    train_loader = loader_dataset(data_list=train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = loader_dataset(data_list=valid_dataset, batch_size=batch_size, shuffle=False) 
    test_loader = loader_dataset(data_list=test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, valid_loader, test_loader

`dataset_name` : 불러올 데이터셋의 이름 입력

`data_type` : Chapter2 에서 저장한 데이터셋 유형 (Token, Fingerprint, Descriptor, Graph) 중 하나를 선택

`batch_size` : 한번에 처리할 샘플 개수를 지정

 Graph loader가 어떻게 구성되어 있는지 확인해본다

### Graph Loader

In [5]:
train_loader, valid_loader, test_loader = get_dataloader(dataset_name = 'bace', data_type='Graph', batch_size = 128)
for data in train_loader:
    print(data)
    break

DataBatch(x=[4209, 9], edge_index=[2, 9110], edge_attr=[9110, 3], smiles=[128], y=[128, 1], batch=[4209], ptr=[129])


- x: 그래프의 노드 feature
- edge_index: 그래프의 두 노드 간 연결(edge) 정보
- edge_attr : 노드 간 연결(edge)의 feature
- smiles : 그래프가 표현하는 분자의 smiles
- y : task에 대한 label
- batch : batch 내의 총 노드 개수
- ptr : 각 그래프의 시작 노드의 위치. 그래프 개수(batch size) + 1 의 길이이며, 마지막 값은 batch에 포함된 전체 노드의 개수를 의미.

다른 표현형들의 경우 feature는 x에, label은 y에 저장되어있다.

## 2) 모델 제작

분자의 표현형에 따라 데이터 타입이 달라지므로, 각기 다른 모델을 사용하여야 한다.

예를 들어 Graph 형태의 데이터는 Graph를 처리할 수 있는 GNN을, String Tokenization 형태의 경우 순서 정보를 반영할 수 있는 1D CNN, RNN 등을 사용할 수 있다.

가장 기본적인 모델인 MLP를 먼저 사용해보자


1. Fingerprint

- Fingerprint는 분자의 특성을 단순히 vector 형태로 나타낸 것이다.
- 따라서 기본 모델인 MLP로 학습하는 것이 적합하다.
- 아래에 정의한 MLP 차원 및 layer 개수는 예시이므로, 데이터셋에 적합하게 수정하여 사용한다.

In [6]:
# 모델 
class MLP(nn.Module): 
    def __init__(self, input_dim, hidden_dim, num_classes, dropout_rate=0.5): 
        super(MLP, self).__init__() 
        self.layer1 = nn.Linear(input_dim, hidden_dim) # input feature가 4개 
        self.layer2 = nn.Linear(hidden_dim,hidden_dim*2)
        self.layer3 = nn.Linear(hidden_dim*2, num_classes) 
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        
    def forward(self,data): # 모델에서 실행되어야 하는 계산을 정의
        x = self.layer1(data.x.float()) # data의 feature가 들어있는 x 만 받아옴, 연산 시 데이터 타입을 일치시키기 위해 float() 
        x = self.relu(x)
        x = self.dropout(x)  
        x = self.layer2(x)
        x = self.relu(x)
        x = self.dropout(x)  
        x = self.layer3(x)   # 출력층
        return x

- `input dim` : 모델의 입력 차원을 의미하며, 데이터 feature의 차원에 따라 결정되므로 적용하려는 데이터에 맞게 설정한다. 

- `hidden dim` : 입력 층을 거친 후 은닉층의 차원을 설정하는 부분이다. 이 파라미터와 layer 개수에 따라 모델의 capacity가 결정된다.

- `num_classes` : 분류할 클래스의 개수를 나타낸다.

- `dropout_rate` : dropout은 일부 뉴런을 랜덤으로 비활성화 하여 과적합을 방지하기 위한 기법이다. 이 파라미터는 비활성화 할 뉴런의 비율을 결정한다.

- 각 레이어 사이에 비선형성을 추가하기 위해 활성화 함수가 포함된다.(예시에서는 ReLU사용)




학습 전, 입력 차원을 결정하기 위해 data loader에서 모델에 학습할 정보를 담고 있는 x의 shape를 확인한다.

In [7]:
# Fingerprint Loader 확인
train_loader, valid_loader, test_loader = get_dataloader(dataset_name = 'bace', data_type='Fingerprint', batch_size = 128)
for data in train_loader:
    print(data)
    break

DataBatch(x=[128, 1024], y=[128, 1], smiles=[128], batch=[128], ptr=[129])


데이터의 입력을 나타내는 x의 차원이 1024이므로 `input_dim` = 1024가 된다. 또한 bace는 이진 분류이므로 `num_classes` = 1로 설정할 수 있다.

In [8]:
fp_model1 = MLP(input_dim=1024, hidden_dim = 128, num_classes = 1, dropout_rate=0.5)
fp_model1

MLP(
  (layer1): Linear(in_features=1024, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=256, bias=True)
  (layer3): Linear(in_features=256, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
)

임의로 설정된 `hidden_dim` 및 `dropout_rate`을 자유롭게 설정하여 모델을 만들어보자

In [9]:
fp_model2 = MLP(input_dim=1024, hidden_dim = 32, num_classes = 1, dropout_rate=0.7)
fp_model2

MLP(
  (layer1): Linear(in_features=1024, out_features=32, bias=True)
  (layer2): Linear(in_features=32, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.7, inplace=False)
)

2. Descriptors

- Descriptors 역시 분자의 특성을 vector 형태로 나타낸 것이므로 MLP를 사용하여 학습한다.
- 사전 정의한 MLP 모델에서 입력 차원만을 변경하여 사용한다.

In [10]:
# Descriptors Loader 확인
train_loader, valid_loader, test_loader = get_dataloader(dataset_name = 'bace', data_type='Descriptors', batch_size = 128)
for data in train_loader:
    print(data)
    break

DataBatch(x=[128, 210], y=[128, 1], smiles=[128], batch=[128], ptr=[129])


descriptor의 x 차원은 210이므로 `input_dim` = 210이 된다.

In [11]:
ds_model = MLP(input_dim=210, hidden_dim = 128, num_classes = 1, dropout_rate=0.2)
ds_model

MLP(
  (layer1): Linear(in_features=210, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=256, bias=True)
  (layer3): Linear(in_features=256, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
)

3. Token

- Token은 분자를 문자열로 나타낸 데이터 형태이다.
- 따라서 문자열의 순서를 반영할 수 있는 1D CNN을 사용하여 학습해본다.

In [12]:
class Conv1d(nn.Module):
    def __init__(self, vocab_num, emb_dim, hidden_size, kernel_size, num_classes):
        super(Conv1d, self).__init__()
        
        # Embedding Layer
        self.embedding = nn.Embedding(vocab_num, emb_dim)
        
        # Conv1d Layers
        self.layer1 = nn.Sequential(
            nn.Conv1d(in_channels=emb_dim, out_channels=hidden_size, kernel_size=kernel_size, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv1d(in_channels=hidden_size, out_channels=hidden_size // 2, kernel_size=kernel_size, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv1d(in_channels=hidden_size // 2, out_channels=hidden_size // 4, kernel_size=kernel_size, stride=1),
            nn.ReLU(),
            nn.AdaptiveMaxPool1d(1)  # Ensure fixed output size
        )
        
        # Fully Connected Layer
        self.fc = nn.Linear(hidden_size // 4, num_classes)
        nn.init.xavier_uniform_(self.fc.weight)  # Xavier initialization

    def forward(self, data):
        # Embedding Layer
        x = self.embedding(data.x)  # Input shape: (batch_size, seq_len)
        
        # Permute to match Conv1d input format
        x = x.permute(0, 2, 1)  # (batch_size, seq_len, emb_dim) -> (batch_size, emb_dim, seq_len)
        
        # Conv1d Layers
        x = self.layer1(x)  # (batch_size, hidden_size, new_seq_len)
        x = self.layer2(x)  # (batch_size, hidden_size // 2, smaller_seq_len)
        x = self.layer3(x)  # (batch_size, hidden_size // 4, 1)
        
        # Flatten
        x = x.view(x.size(0), -1)  # (batch_size, hidden_size // 4)
        
        # Fully Connected Layer
        x = self.fc(x)  # (batch_size, 1)
        
        return x

- `vocab_num` : 문자열을 구성하는 단어의 종류를 입력한다. chapter2에서 string tokenization 단계에서 출력한 단어집에서 단어의 종류를 알 수 있다. bace의 경우 Unkown 포함 39개가 있다.

- `emb_dim` : 입력된 단어가 임베딩 될 차원을 결정한다.

- `hidden_dim` : 임베딩 된 단어를 학습 시킬 때 은닉층의 차원을 설정한다.

- `kernel_size` : kernel이 한 번에 읽을 시퀀스의 길이를 정의하며, 작을 수록 세밀한 패턴을, 클 수록 넓은 문맥을 포착한다.

- `num_classes` : 분류할 클래스의 개수를 나타낸다.

In [13]:
# Token Loader 확인
train_loader, valid_loader, test_loader = get_dataloader(dataset_name = 'bace', data_type='Token', batch_size = 128)
for data in train_loader:
    print(data)
    break

DataBatch(x=[128, 178], y=[128, 1], smiles=[128], batch=[128], ptr=[129])


In [14]:
model = Conv1d(vocab_num=39, emb_dim=128, hidden_size = 128, kernel_size = 1, num_classes=1)
model

Conv1d(
  (embedding): Embedding(39, 128)
  (layer1): Sequential(
    (0): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv1d(128, 64, kernel_size=(1,), stride=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv1d(64, 32, kernel_size=(1,), stride=(1,))
    (1): ReLU()
    (2): AdaptiveMaxPool1d(output_size=1)
  )
  (fc): Linear(in_features=32, out_features=1, bias=True)
)

4. Graph

- Graph는 분자를 구성하는 각 원자를 노드(node), 원자들의 결합을 엣지(edge)로 나타낸 데이터 형태이다.
- Graph 형태의 데이터를 처리할 수 있는 신경망인 Graph Neural Network(GNN)을 사용하여야 한다.
- 또한 GNN은 GCN, GIN, GAT 등 다양한 종류가 있으므로 하나씩 다루어보도록 한다.

- GNN의 입력에는 노드의 특징 차원이 필요하다. data loader를 print하여 노드 x의 feature 차원을 출력해보자

In [15]:
# Descriptors Loader 확인
train_loader, valid_loader, test_loader = get_dataloader(dataset_name = 'bace', data_type='Graph', batch_size = 128)
for data in train_loader:
        print(data)
        break

DataBatch(x=[4316, 9], edge_index=[2, 9298], edge_attr=[9298, 3], smiles=[128], y=[128, 1], batch=[4316], ptr=[129])


- GNN은 각 노드들의 정보 전파 및 종합이 일어나는 gnn layer와, gnn layer가 생성한 표현을 바탕으로 분류 및 회귀 작업이 일어나는 feed forward network(ffn)로 구성된다. 따라서 `hidden_dim`은 gnn의 은닉층 차원을, `ffn_hidden`은 ffn layer의 은닉층의 차원을 지정하는 파라미터이다.
- `input_dim` : 노드 feature의 차원을 입력한다.
- `num_classes`: ffn layer의 마지막 차원을 결정하는 파라미터이므로, 분류하고자 하는 클래스의 개수를 입력한다.

In [16]:
class GCNNet(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim = 64, ffn_hidden=64, num_classes=1, dropout_rate=0.2):

        super(GCNNet, self).__init__()

        # SMILES graph branch
        self.num_classes = num_classes
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim*2)
        # self.conv3 = GCNConv(hidden_dim*2, hidden_dim * 4)
        # self.fc_g1 = torch.nn.Linear(hidden_dim*4, hidden_dim*2)
        self.fc_g1 = torch.nn.Linear(hidden_dim*2, hidden_dim)
        self.fc_g2 = torch.nn.Linear(hidden_dim, num_classes)

        # activation
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, data):
        # get graph input
        x, edge_index, batch = data.x.float(), data.edge_index, data.batch

        x = self.conv1(x, edge_index)
        x = self.relu(x)

        x = self.conv2(x, edge_index)
        x = self.relu(x)

        # x = self.conv3(x, edge_index)
        # x = self.relu(x)
        x = gmp(x, batch)       # global max pooling

        # ffn layer
        x = self.fc_g1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc_g2(x)
        out = self.dropout(x)
        
        return out

In [17]:
g_model1 = GCNNet(input_dim=9)
g_model1

GCNNet(
  (conv1): GCNConv(9, 64)
  (conv2): GCNConv(64, 128)
  (fc_g1): Linear(in_features=128, out_features=64, bias=True)
  (fc_g2): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
)

In [18]:
class GATNet(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim = 64, ffn_hidden=128, num_classes=1, dropout_rate=0.2):
        super(GATNet, self).__init__()

        # graph layers
        self.gcn1 = GATConv(input_dim, hidden_dim, heads=10, dropout=dropout_rate)
        self.gcn2 = GATConv(hidden_dim*10, hidden_dim*2, dropout=dropout_rate)
        self.fc_g1 = nn.Linear(hidden_dim*2, ffn_hidden)
        self.fc_g2 = nn.Linear(ffn_hidden, num_classes)

        # activation and regularization
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, data):
        # graph input feed-forward
        x, edge_index, batch = data.x.float(), data.edge_index, data.batch

        x = F.dropout(x, p=0.2, training=self.training)
        x = F.elu(self.gcn1(x, edge_index))
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.gcn2(x, edge_index)
        x = self.relu(x)
        x = gmp(x, batch)          # global max pooling

        # ffn layer
        x = self.fc_g1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc_g2(x)
        out = self.dropout(x)
        
        return out

In [19]:
g_model2 = GATNet(input_dim=9)
g_model2

GATNet(
  (gcn1): GATConv(9, 64, heads=10)
  (gcn2): GATConv(640, 128, heads=1)
  (fc_g1): Linear(in_features=128, out_features=128, bias=True)
  (fc_g2): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
)

In [20]:
class GINConvNet(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim = 128, ffn_hidden=128, num_classes=1, dropout_rate=0.2):
        super(GINConvNet, self).__init__()

        dim = 32
        self.dropout = nn.Dropout(dropout_rate)
        self.relu = nn.ReLU()
        self.num_classes = num_classes
        # convolution layers
        nn1 = Sequential(Linear(input_dim, dim), ReLU(), Linear(dim, dim))
        self.conv1 = GINConv(nn1)
        self.bn1 = torch.nn.BatchNorm1d(dim)

        nn2 = Sequential(Linear(dim, dim), ReLU(), Linear(dim, dim))
        self.conv2 = GINConv(nn2)
        self.bn2 = torch.nn.BatchNorm1d(dim)

        nn3 = Sequential(Linear(dim, dim), ReLU(), Linear(dim, dim))
        self.conv3 = GINConv(nn3)
        self.bn3 = torch.nn.BatchNorm1d(dim)

        nn4 = Sequential(Linear(dim, dim), ReLU(), Linear(dim, dim))
        self.conv4 = GINConv(nn4)
        self.bn4 = torch.nn.BatchNorm1d(dim)

        nn5 = Sequential(Linear(dim, dim), ReLU(), Linear(dim, dim))
        self.conv5 = GINConv(nn5)
        self.bn5 = torch.nn.BatchNorm1d(dim)

        self.fc = Linear(dim, hidden_dim)
        
        self.fc_g1 = nn.Linear(hidden_dim, ffn_hidden)
        self.fc_g2 = nn.Linear(ffn_hidden, self.num_classes)
    

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        x = F.relu(self.conv1(x, edge_index))
        x = self.bn1(x)
        x = F.relu(self.conv2(x, edge_index))
        x = self.bn2(x)
        x = F.relu(self.conv3(x, edge_index))
        x = self.bn3(x)
        x = F.relu(self.conv4(x, edge_index))
        x = self.bn4(x)
        x = F.relu(self.conv5(x, edge_index))
        x = self.bn5(x)
        
        x = global_add_pool(x, batch)
        x = F.relu(self.fc(x))
        x = F.dropout(x, p=0.2, training=self.training)

        # add some dense layers
        x = self.fc_g1(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc_g2(x)
        x = self.relu(x)
        out = self.dropout(x)
        
        return out

In [21]:
g_model3 = GINConvNet(input_dim=9)
g_model3

GINConvNet(
  (dropout): Dropout(p=0.2, inplace=False)
  (relu): ReLU()
  (conv1): GINConv(nn=Sequential(
    (0): Linear(in_features=9, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
  ))
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): GINConv(nn=Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
  ))
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): GINConv(nn=Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
  ))
  (bn3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): GINConv(nn=Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features

## 3) 학습 및 평가


In [22]:
# 학습 진행 함수 정의
device = "cuda" if torch.cuda.is_available() else "cpu"

def train(model, train_loader, valid_loader, epochs):
    model = model.to(device)
    # train_loader, valid_loader, test_loader = loaders
    optimizer = torch.optim.SGD(model.parameters(), lr = 0.00001, momentum = 0.5) 
    criterion = nn.BCEWithLogitsLoss() 

    best_valid_loss = float('inf')  # 초기값을 무한대로 설정
    best_epoch = 0  # 가장 성능 좋은 epoch을 기록

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        train_loss = 0.0
        model.train() # 학습 모드 전환    

        # train
        for data in train_loader:
            data = data.to(device)
            y = data.y
            
            optimizer.zero_grad() # 전 단계에서의 loss gradient 값을 초기화
            output = model(data)
            loss = criterion(output,y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        print(f"Training Loss: {train_loss / len(train_loader):.4f}")

        # validation
        model.eval() # 학습 모드 전환    
        valid_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():  # Gradient 계산 비활성화
            for data in valid_loader:
                data = data.to(device)
                y = data.y
                
                output = model(data)
                loss = criterion(output, y)
                valid_loss += loss.item()

                # Accuracy 계산
                predicted = (F.sigmoid(output) > 0.5).int()

                total += y.size(0)
                correct += (predicted == y).sum().item()

        accuracy = 100 * (correct / total)
        avg_valid_loss = valid_loss / len(valid_loader)
        print(f"Validation Loss: {avg_valid_loss:.4f}, Validation Accuracy: {accuracy:.2f}%\n")
        
        # Best model 저장
        if avg_valid_loss < best_valid_loss:
            best_valid_loss = avg_valid_loss
            best_epoch = epoch + 1
            torch.save(model.state_dict(), "best_epoch.pth")
            print(f"Best model updated at epoch {best_epoch} with validation loss: {best_valid_loss:.4f}")

    print(f"Training completed. Best model was at epoch {best_epoch} with validation loss: {best_valid_loss:.4f}")
    

- `model` : 학습할 데이터셋에 맞게 설정한 모델 입력
- `train_loader`, `valid_loader` : 학습, 검증 data loader 입력
- `epochs` : 반복할 학습 횟수 입력

In [23]:
# fingerprint loader 생성
train_loader, valid_loader, test_loader = get_dataloader(dataset_name = 'bace', data_type='Fingerprint', batch_size = 128)

# 학습할 모델 정의
fp_model1 = MLP(input_dim=1024, hidden_dim = 128, num_classes = 1, dropout_rate=0.5)

# train
p = train(fp_model1, train_loader, valid_loader, epochs=10)

Epoch 1/10
Training Loss: 0.7022
Validation Loss: 0.6539, Validation Accuracy: 86.09%

Best model updated at epoch 1 with validation loss: 0.6539
Epoch 2/10
Training Loss: 0.7028


/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Validation Loss: 0.6539, Validation Accuracy: 86.09%

Epoch 3/10
Training Loss: 0.7024
Validation Loss: 0.6539, Validation Accuracy: 86.09%

Epoch 4/10
Training Loss: 0.7037
Validation Loss: 0.6539, Validation Accuracy: 86.09%

Epoch 5/10
Training Loss: 0.7024
Validation Loss: 0.6539, Validation Accuracy: 86.09%

Epoch 6/10
Training Loss: 0.7021
Validation Loss: 0.6540, Validation Accuracy: 86.09%

Epoch 7/10
Training Loss: 0.7012
Validation Loss: 0.6540, Validation Accuracy: 86.09%

Epoch 8/10
Training Loss: 0.7011
Validation Loss: 0.6540, Validation Accuracy: 86.09%

Epoch 9/10
Training Loss: 0.7017
Validation Loss: 0.6540, Validation Accuracy: 86.09%

Epoch 10/10
Training Loss: 0.7024
Validation Loss: 0.6540, Validation Accuracy: 86.09%

Training completed. Best model was at epoch 1 with validation loss: 0.6539


In [24]:
# Descriptor loader 생성
train_loader, valid_loader, test_loader = get_dataloader(dataset_name = 'bace', data_type='Descriptors', batch_size = 128)

# 학습할 모델 정의
ds_model = MLP(input_dim=210, hidden_dim = 128, num_classes = 1, dropout_rate=0.7)

# train
train(ds_model, train_loader, valid_loader, epochs=10)

Epoch 1/10
Training Loss: 0.6944
Validation Loss: 0.7093, Validation Accuracy: 34.44%

Best model updated at epoch 1 with validation loss: 0.7093
Epoch 2/10
Training Loss: 0.6977
Validation Loss: 0.7094, Validation Accuracy: 34.44%

Epoch 3/10


/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Training Loss: 0.6951
Validation Loss: 0.7094, Validation Accuracy: 34.44%

Epoch 4/10
Training Loss: 0.7051
Validation Loss: 0.7095, Validation Accuracy: 34.44%

Epoch 5/10
Training Loss: 0.7032
Validation Loss: 0.7096, Validation Accuracy: 34.44%

Epoch 6/10
Training Loss: 0.6993
Validation Loss: 0.7096, Validation Accuracy: 33.77%

Epoch 7/10
Training Loss: 0.7085
Validation Loss: 0.7097, Validation Accuracy: 33.77%

Epoch 8/10
Training Loss: 0.7050
Validation Loss: 0.7097, Validation Accuracy: 33.77%

Epoch 9/10
Training Loss: 0.7028
Validation Loss: 0.7098, Validation Accuracy: 33.77%

Epoch 10/10
Training Loss: 0.7012
Validation Loss: 0.7098, Validation Accuracy: 33.77%

Training completed. Best model was at epoch 1 with validation loss: 0.7093


In [25]:
# Token loader 생성
train_loader, valid_loader, test_loader = get_dataloader(dataset_name = 'bace', data_type='Token', batch_size = 128)

# 학습할 모델 정의
token_model = Conv1d(vocab_num=39, emb_dim=128, hidden_size = 128, kernel_size = 1, num_classes=1)

# train
train(token_model, train_loader, valid_loader, epochs=10)

Epoch 1/10
Training Loss: 0.6774
Validation Loss: 0.9769, Validation Accuracy: 13.91%

Best model updated at epoch 1 with validation loss: 0.9769
Epoch 2/10
Training Loss: 0.6769
Validation Loss: 0.9768, Validation Accuracy: 13.91%

Best model updated at epoch 2 with validation loss: 0.9768
Epoch 3/10
Training Loss: 0.6752
Validation Loss: 0.9767, Validation Accuracy: 13.91%

Best model updated at epoch 3 with validation loss: 0.9767
Epoch 4/10
Training Loss: 0.6778
Validation Loss: 0.9766, Validation Accuracy: 13.91%

Best model updated at epoch 4 with validation loss: 0.9766
Epoch 5/10
Training Loss: 0.6742
Validation Loss: 0.9766, Validation Accuracy: 13.91%

Best model updated at epoch 5 with validation loss: 0.9766
Epoch 6/10
Training Loss: 0.6762
Validation Loss: 0.9765, Validation Accuracy: 13.91%

Best model updated at epoch 6 with validation loss: 0.9765
Epoch 7/10
Training Loss: 0.6778
Validation Loss: 0.9764, Validation Accuracy: 13.91%

Best model updated at epoch 7 with va

In [26]:
# Graph loader 생성
train_loader, valid_loader, test_loader = get_dataloader(dataset_name = 'bace', data_type='Graph', batch_size = 128)

# 학습할 모델 정의
g_model3 = GINConvNet(input_dim=9)

# train
train(g_model3, train_loader, valid_loader, epochs=10)

Epoch 1/10
Training Loss: 0.9844
Validation Loss: 0.5003, Validation Accuracy: 86.09%

Best model updated at epoch 1 with validation loss: 0.5003
Epoch 2/10
Training Loss: 0.9103
Validation Loss: 0.3343, Validation Accuracy: 86.09%

Best model updated at epoch 2 with validation loss: 0.3343
Epoch 3/10
Training Loss: 0.9384
Validation Loss: 0.3036, Validation Accuracy: 86.09%

Best model updated at epoch 3 with validation loss: 0.3036
Epoch 4/10
Training Loss: 0.8880
Validation Loss: 0.3002, Validation Accuracy: 85.43%

Best model updated at epoch 4 with validation loss: 0.3002
Epoch 5/10
Training Loss: 0.8842
Validation Loss: 0.3285, Validation Accuracy: 80.79%

Epoch 6/10
Training Loss: 0.8785
Validation Loss: 0.3714, Validation Accuracy: 71.52%

Epoch 7/10
Training Loss: 0.8923
Validation Loss: 0.3800, Validation Accuracy: 69.54%

Epoch 8/10
Training Loss: 0.9000
Validation Loss: 0.4012, Validation Accuracy: 68.87%

Epoch 9/10
Training Loss: 0.8707
Validation Loss: 0.4213, Validation

학습된 모델로 평가를 수행한다.

평가는 accuracy, precision, recall, f1_score, roc_auc, average precision 총 6가지로 이루어지며, 설명은 다음과 같다.

- Accuracy (정확도): 올바르게 예측한 샘플의 비율

- Precision (정밀도): 모델이 양성으로 예측한 샘플 중 실제로 양성인 비율.

- Recall (재현율): 실제 양성 샘플 중 모델이 양성으로 예측한 비율.

- F1-Score: Precision과 Recall의 조화 평균.데이터가 불균형할 때 모델의 전반적 성능을 파악하기 좋음.

- AUC_ROC (ROC 곡선 아래 면적): 민감도(TPR)와 특이도(1-FPR) 간의 균형을 평가.1에 가까울수록 좋음.

- AUC_PRC (PRC 곡선 아래 면적): Precision과 Recall 간의 관계를 시각화한 곡선의 면적.

In [27]:
def test(model, test_loader, save_file = 'Results.csv'):
    model = model.to(device)
    model.eval()    
    criterion = nn.BCEWithLogitsLoss() 
    
    all_preds = []
    all_targets = []
    test_loss = 0.0
    
    with torch.no_grad():  # Gradient 계산 비활성화
        for data in test_loader:
            data = data.to(device)
            y = data.y
            
            output = model(data)
            preds = torch.sigmoid(output).cpu().numpy().ravel()  # Sigmoid로 확률 변환 후 numpy로 변환
            all_preds.append(preds)
            all_targets.append(y.cpu().numpy())
            
            # 손실 계산
            test_loss += criterion(output, y).item()
            
        # Concatenate predictions and targets
        all_preds = np.concatenate(all_preds)
        all_targets = np.concatenate(all_targets).astype(int)
    
        # 메트릭 계산
        test_acc = accuracy_score(all_targets, (all_preds >= 0.5).astype(int))
        test_precision = precision_score(all_targets, (all_preds >= 0.5).astype(int))
        test_recall = recall_score(all_targets, (all_preds >= 0.5).astype(int))
        test_f1 = f1_score(all_targets, (all_preds >= 0.5).astype(int))
        test_auc_roc = roc_auc_score(all_targets, all_preds)
        test_auc_prc = average_precision_score(all_targets, all_preds)
    
        # 평균 손실 계산
        test_loss /= len(test_loader)
    
        # 결과 저장
        metrics = {
            "Loss": test_loss,
            "Accuracy": test_acc,
            "Precision": test_precision,
            "Recall": test_recall,
            "F1-Score": test_f1,
            "AUC_ROC": test_auc_roc,
            "AUC_PRC": test_auc_prc
        }
    
        # 결과 출력
        print("\n최적 모델의 테스트 데이터 성능:")
        print(tabulate(pd.DataFrame(metrics, index=["Metric Value"]).T, headers="keys", tablefmt="fancy_grid"))
    
        # CSV 파일 저장
        df = pd.DataFrame(metrics, index=[0])
        df.to_csv(save_file, index=False)
        print(f"\nResults saved to {save_file}")

    return metrics

In [28]:
# best model load
g_model3 = GINConvNet(input_dim=9)
g_model3.load_state_dict(torch.load("best_epoch.pth"))
test(g_model3, test_loader)


최적 모델의 테스트 데이터 성능:
╒═══════════╤════════════════╕
│           │   Metric Value │
╞═══════════╪════════════════╡
│ Loss      │       0.548003 │
├───────────┼────────────────┤
│ Accuracy  │       0.532895 │
├───────────┼────────────────┤
│ Precision │       0.532895 │
├───────────┼────────────────┤
│ Recall    │       1        │
├───────────┼────────────────┤
│ F1-Score  │       0.695279 │
├───────────┼────────────────┤
│ AUC_ROC   │       0.663537 │
├───────────┼────────────────┤
│ AUC_PRC   │       0.671833 │
╘═══════════╧════════════════╛

Results saved to Results.csv


{'Loss': 0.5480027794837952,
 'Accuracy': 0.5328947368421053,
 'Precision': 0.5328947368421053,
 'Recall': 1.0,
 'F1-Score': 0.6952789699570815,
 'AUC_ROC': 0.6635367762128326,
 'AUC_PRC': 0.6718333041640333}